In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
%sql
MERGE INTO ${catalog}.${schema}.taxhistory_silver as target
USING (
  SELECT *
  FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY zpid, year ORDER BY file_name DESC) AS rn
    FROM ${catalog}.${schema}.tax_bronze
  ) AS deduped
  WHERE rn = 1
) as source
ON target.zpid = source.zpid AND target.year = source.year

WHEN MATCHED THEN 
  UPDATE SET 
    target.value = source.value,
    target.taxPaid = source.taxPaid,
    target.valueIncreaseRate = source.valueIncreaseRate,
    target.taxIncreaseRate = source.taxIncreaseRate,
    target.file_name = source.file_name,
    target.Md5_Hash = md5(concat_ws('', source.zpid, source.year, source.value, source.taxPaid, source.valueIncreaseRate, source.taxIncreaseRate, source.file_name))

WHEN NOT MATCHED THEN 
  INSERT (
    zpid,
    year,
    value,
    taxPaid,
    valueIncreaseRate,
    taxIncreaseRate,
    file_name,
    Md5_Hash
  )
  VALUES (
    source.zpid,
    source.year,
    source.value,
    source.taxPaid,
    source.valueIncreaseRate,
    source.taxIncreaseRate,
    source.file_name,
    md5(concat_ws('', source.zpid, source.year, source.value, source.taxPaid, source.valueIncreaseRate, source.taxIncreaseRate, source.file_name))
  )


In [0]:
%sql
select * from ${catalog}.${schema}.taxhistory_silver